In [1]:
# Disable jedi autocompleter
%config Completer.use_jedi = False

In [1]:
%%writefile requirements_custom.txt
transformers==4.6.1
torch==1.8.1
torchvision==0.9.1
pandas==1.1.5
sklearn==0.0
matplotlib==3.4.2
ipywidgets==7.6.3
datasets==1.6.2
seqeval==1.2.2

Overwriting requirements_custom.txt


In [2]:
!pip install -r requirements_custom.txt

     |████████████████████████████████| 2.2 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 119.6 MB 1.9 MB/s eta 0:00:01     |████████████████████▊           | 77.5 MB 943 kB/s eta 0:00:45     |███████████████████████▍        | 87.3 MB 2.4 MB/s eta 0:00:14
     |████████████████████████████████| 13.2 MB 925 kB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 370 kB/s eta 0:00:01
     |████████████████████████████████| 121 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 221 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 91 kB/s eta 0:00:011
     |████████████████████████████████| 2.2 MB 348 kB/s eta 0:00:01
     |████████████████████████████████| 284 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 16.0 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 75 k

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
from pathlib import Path
from sklearn.model_selection import train_test_split

# NER BETO

A modo de ejemplo, usaremos la implementación mostrada en [Token Classification](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb), ajustada al caso de BETO [publicación asociada](https://users.dcc.uchile.cl/~jperez/papers/pml4dc2020.pdf). Otro ejemplo interesante a revisar es [TinyBERT](https://huggingface.co/mrm8488/TinyBERT-spanish-uncased-finetuned-ner)

In [4]:
import torch
# from transformers import BertForMaskedLM, BertTokenizer
from transformers import BertForTokenClassification

In [5]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "dccuchile/bert-base-spanish-wwm-uncased"
batch_size = 16

Definimos el dispositivo sobre el cual se hará entrenamiento, en caso de tener disponible una GPU

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
model = BertForTokenClassification.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-ba

Podemos pasar el modelo al dispositivo asociado, y ver la arquitectura pre-entrenada

In [8]:
model.to(device)
model.train()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

## Carga de dataset

In [9]:
from datasets import load_dataset, load_metric, list_datasets

In [10]:
datasets_list = list_datasets()
print(', '.join(dataset for dataset in datasets_list))

acronym_identification, ade_corpus_v2, adversarial_qa, aeslc, afrikaans_ner_corpus, ag_news, ai2_arc, air_dialogue, ajgt_twitter_ar, allegro_reviews, allocine, alt, amazon_polarity, amazon_reviews_multi, amazon_us_reviews, ambig_qa, amttl, anli, app_reviews, aqua_rat, aquamuse, ar_cov19, ar_res_reviews, ar_sarcasm, arabic_billion_words, arabic_pos_dialect, arabic_speech_corpus, arcd, arsentd_lev, art, arxiv_dataset, ascent_kb, aslg_pc12, asnq, asset, assin, assin2, atomic, autshumato, babi_qa, banking77, bbaw_egyptian, bbc_hindi_nli, bc2gm_corpus, best2009, bianet, bible_para, big_patent, billsum, bing_coronavirus_query_set, biomrc, blended_skill_talk, blimp, blog_authorship_corpus, bn_hate_speech, bookcorpus, bookcorpusopen, boolq, bprec, break_data, brwac, bsd_ja_en, bswac, c3, c4, cail2018, caner, capes, catalonia_independence, cawac, cbt, cc100, cc_news, ccaligned_multilingual, cdsc, cdt, cfq, chr_en, cifar10, cifar100, circa, civil_comments, clickbait_news_bg, climate_fever, clinc

Usaremos para efectos de NER el dataset ```conll2002``` con los tags para la tarea asociada. Si se desea utilizar un dataset distinto fuera de los de ejemplo, se recomienda revisar el siguiente [enlace](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files)

In [11]:
dataset = load_dataset(
   'conll2002', 'es')

Dataset conll2002 downloaded and prepared to /Users/iacastro/.cache/huggingface/datasets/conll2002/es/1.0.0/a3a8a8612caf57271f5b35c5ae1dd25f99ddb9efb9c1667abaa70ede33e863e5. Subsequent calls will reuse this data.


El dataset es un diccionario que contiene separación para ```train```, ```validation``` y ```test```

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1518
    })
})

Para obtener la representación en string de las clases para los token de NER, se pueden extraer directamente del campo ```features```

In [13]:
dataset["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

Estas entidades están en el formato BIO

* B por "beginning", se refiera a tokens que están al comienzo de cada entidad
* I por "intermediate", se refieren a tokens que están en medio de cada entidad
* O por "outside", se refieren a tokens que no pertenecen a entidades relevantes para el problema en cuestión

Por otro lado, para este dataset específico se tienen las siguientes entidades

* PER para persona
* ORG para organización
* LOC para ubicación
* MISC para misceláneo

De lo anterior podemos obtener sólo la lista con las entidades

In [14]:
label_list = dataset["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

Definamos ahora una función para formatear y cargar elementos aleatorios

In [15]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(dataset["train"])

,id,ner_tags,pos_tags,tokens
0,542,"[O, O, O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[VMI, CS, DA, NC, SP, DA, NC, Fc, CS, DA, NC, SP, DA, NC, SP, NC, Fc, Fe, RN, VAI, VMP, SP, PI, Fe, CC, VMI, CS, SP, RG, Fc, DA, NC, SP, NC, VAI, VSP, DA, RG, AQ, SP, DA, NC, SP, DD, NC, CC, SP, DA, NC, Fc, CC, CS, VMM, VMI, RG, SP, DA, NC, SP, NC, Fp]","[Añadió, que, las, actuaciones, de, la, Administración, ,, como, la, rebaja, de, los, módulos, del, IRPF, ,, "", no, han, valido, para, nada, "", y, explicó, que, hasta, ahora, ,, los, productores, de, secano, han, sido, los, más, perjudicados, por, el, incremento, de, este, insumo, y, de, los, fertilizantes, ,, pero, si, continúa, perjudicará, también, a, las, explotaciones, de, regadío, .]"
1,7914,"[O, O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER, O, O, O, O, B-PER, I-PER, O, O, O, O]","[SP, RG, NC, Fc, DA, NC, SP, DA, NC, Fc, DA, NC, AQ, AQ, Fc, CC, DA, NC, NC, AQ, VMI, NC, AQ, Fp]","[De, igual, forma, ,, el, líder, de, la, montaña, ,, el, holandés, Karsten, Kroon, ,, y, el, español, Angel, Vicioso, sufren, contusiones, varias, .]"
2,7014,"[O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[SP, DA, AO, DN, NC, DA, NC, SP, NC, VMI, SP, NC, AQ, NC, SP, NC, SP, VMN, DI, NC, SP, DA, NC, CC, DI, NC, SP, DA, NC, Fp]","[En, los, últimos, quince, días, los, bomberos, de, Vigo, llevaron, a, cabo, numerosas, acciones, de, protesta, para, reclamar, una, mejora, en, los, medios, y, un, aumento, en, la, plantilla, .]"
3,6324,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, O]","[Fe, VAI, NC, SP, DA, NC, SP, AQ, CC, NC, Fc, RG, DA, AQ, Fc, CC, CS, PI, VMI, NC, AQ, CC, VMI, DP, NC, PP, VMI, AQ, CS, SP, DA, RG, P0, PP, VMS, RG, AQ, Fe, Fc, VMI, NC, Fp]","["", Hay, atrasos, en, los, pagos, a, militares, y, policías, ,, especialmente, los, militares, ,, y, si, uno, tiene, gente, armada, y, pretende, su, lealtad, me, parece, lógico, que, por, lo, menos, se, le, tenga, bien, atendida, "", ,, subrayó, Galaverna, .]"
4,1533,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[PP, Fc, SP, VMM, Fc, SP, DA, NC, SP, DA, NC, AQ, Fpa, Z, SP, PN, Fpt, CC, SP, DA, AQ, Fpa, Z, SP, PN, Fpt, Fc, DA, PR, VMI, DA, NC, AQ, SP, DI, NC, AQ, AQ, CC, DA, NC, SP, NC, VMI, SP, NC, CC, SP, NC, Fp]","[Ello, ,, según, justifica, ,, por, el, alza, de, la, demanda, interna, (, +7,1, por, ciento, ), y, de, la, externa, (, +13,2, por, ciento, ), ,, lo, que, refleja, el, efecto, positivo, de, una, situación, internacional, favorable, y, la, depreciación, del, euro, respeto, del, dólar, y, del, yen, .]"
5,2436,"[O, O, O, O, B-MISC, I-MISC, O, O, O, B-MISC, I-MISC, O, O, O, B-MISC, O, O, O, B-MISC, I-MISC, O, O, O, B-MISC, I-MISC, I-MISC, I-MISC, I-MISC, O, O, B-MISC, I-MISC, I-MISC, I-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MISC, I-MISC, I-MISC, I-MISC, I-MISC, I-MISC, O, O]","[SP, NC, CS, Fe, DP, NC, Fe, Fc, Fe, PP, VMI, Fe, Fc, Fe, NC, Fe, Fc, Fe, NC, AQ, Fe, Fc, Fe, PP, VMI, DI, NC, AQ, Fe, Fe, DA, NC, DA, NC, Fe, Fc, SP, RG, SP, DN, NC, Fc, DA, NC, SP, PP, NC, Fc, DA, NC, VMI, DI, NC, SP, DA, NC, PR, VMI, DA, NC, AQ, SP, DA, NC, SP, NC, Z, Fc, SP, DA, NC, Fe, AQ, NC, SP, DA, NC, AQ, Fe, Fp]","[Con, canciones, como, "", Mi, Jaca, "", ,, "", La, parrala, "", ,, "", Tatuaje, "", ,, "", Raska, yu, "", ,, "", Yo, vendo, unos, ojos, negros, "", "", La, luna., el, toro, "", ,, hasta, más, de, cuatrocientas, composiciones, ,, la, mayoría, de, ellas, coplas, ,, el, autor, hace, un, inventario, de, las, canciones, que, alimentaron, el, gusto, popular, durante, la, postguerra, hasta, llegar., 1975, 

## Preprocesamiento de datos

Primero tokenizamos el texto que le ingestaremos al modelo según el formato que éste espera para el entrenamiento

Para tener esto, instanciaremos el tokenizador con la función ```AutoTokenizer.from_pretrained```, la cual permite:

* Obtener un tokenizador que corresponda a la arquitectura de modelo que usaremos
* Descargar el vocabulario utilizado cuando se pre-entrenó este checkpoint de modelo en particular

El vocabulario se guarda en caché, de manera de no descargarlo nuevamente cuando se ejecute la celda.

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Validamos si el modelo tiene un tokenizador rápido mantenido por librerías (es necesario en este notebook)

In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

A continuación, para probar su funcionamiento se tomará una sentencia de prueba que nos permitirá ver como llamar al tokenizador

In [19]:
test_sentence = "¡Hola, esta es una oración!"

Primero para un string completo

In [20]:
tokenizer(test_sentence)

{'input_ids': [4, 1120, 1734, 1019, 1149, 1028, 1091, 9301, 1109, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

O como en este caso, separado por palabras/tokens

In [21]:
tokenizer(test_sentence.split(), is_split_into_words=True)

{'input_ids': [4, 1120, 1734, 1019, 1149, 1028, 1091, 9301, 1109, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Tomemos ahora un texto de ejemplo desde nuestro conjunto de entrenamiento con los tokens. La idea aquí es validar si el modelo posee tokenizado sub palabra

In [22]:
example = dataset["train"][5]
print(example["tokens"])
print(example["ner_tags"])

['Por', 'su', 'parte', ',', 'el', 'Abogado', 'General', 'de', 'Victoria', ',', 'Rob', 'Hulls', ',', 'indicó', 'que', 'no', 'hay', 'nadie', 'que', 'controle', 'que', 'las', 'informaciones', 'contenidas', 'en', 'CrimeNet', 'son', 'veraces', '.']
[0, 0, 0, 0, 0, 1, 2, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]


In [23]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'por', 'su', 'parte', ',', 'el', 'abogado', 'general', 'de', 'victoria', ',', 'rob', 'hul', '##ls', ',', 'indicó', 'que', 'no', 'hay', 'nadie', 'que', 'control', '##e', 'que', 'las', 'informaciones', 'contenidas', 'en', 'crimen', '##et', 'son', 'vera', '##ces', '.', '[SEP]']


Se observa que la palabra "hulls" se dividió en 2 subtokens, también "crimenet" y "veraces"

De lo anterior se ve que se requiere un preprocesamiento para manejar esos subtokens, además de caracteres especiales como ```[CLS]``` o ```[SEP]```. Podemos ver esto si comparamos los tags originales contra los tokenizados en el largo de cada uno

In [24]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(29, 35)

Vemos que se generaron algunos tokens adicionales. Para recuperarlos, podemos usar los ID de tokenizado

In [25]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 27, 28, None]


Acá vemos que los caracteres especiales se identifican como ```None```

In [26]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

35 35


Aquí se colocan los token especiales como -100 (índice ignorado por PyTorch) y las etiquetas de todos los demás tokens por la etiqueta de la palabra sobre la cual se originan. Otra estrategia es colocar la etiqueta al primer token de una palabra, y luego -100 a todos los subtokens obtenidos de la misma. En el código a continuación se ponen ambos enfoques, separados por el flag ```label_all_tokens``` (```True``` para primer enfoque y ```False``` para el segundo respectivamente)

In [27]:
label_all_tokens = True

In [28]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

La función permite procesar múltiples oraciones a la vez

In [29]:
tokenize_and_align_labels(dataset['train'][:5])

{'input_ids': [[4, 26890, 1147, 6154, 1135, 1019, 2581, 1533, 1147, 1818, 30955, 1135, 1008, 5], [4, 1139, 5], [4, 1039, 4682, 1661, 1081, 1520, 1019, 30374, 1005, 12103, 1019, 17774, 2066, 1032, 3503, 1009, 2736, 2235, 1097, 4190, 1074, 2074, 5957, 20169, 2767, 1012, 1091, 4448, 1009, 4730, 1041, 4153, 9570, 1503, 1039, 5333, 1009, 1067, 3551, 7412, 1009, 1032, 2030, 1008, 5], [4, 1032, 7078, 1081, 4682, 1661, 1394, 1646, 1649, 1009, 1041, 1044, 5804, 1081, 3151, 9396, 1081, 1520, 1009, 4848, 1147, 6154, 1135, 1057, 15969, 17892, 1012, 27032, 1044, 8573, 3557, 1040, 16203, 1039, 2621, 2306, 1039, 11717, 1009, 1032, 4019, 1009, 1041, 1085, 1845, 1041, 1086, 5060, 1494, 7371, 1887, 10816, 1926, 1246, 1039, 9029, 1012, 2589, 1009, 1032, 4448, 5728, 3283, 1008, 5], [4, 1149, 4448, 1005, 3686, 3664, 1044, 1935, 1009, 5386, 1019, 1526, 1035, 1039, 1041, 1075, 1764, 5016, 1167, 1035, 1186, 1009, 1044, 7098, 1009, 5996, 1019, 1040, 15411, 1926, 1246, 5015, 1009, 8490, 1040, 10938, 1319, 13201

Para aplicar la función a todo el dataset, usamos ```map``` para aplicar sobre ```train```, ```validation``` y ```test``` en una sola operación

In [30]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

## Ajuste fino de modelo

Con los datos listos, descargamos el modelo pre-entrenado y lo ajustamos. Dado que todas las tareas a realizar son para clasificación de tokens, usamos la clase ```AutoModelForTokenClassification```. Como con el tokenizado, usamos la función ```from_pretrained``` para descargar y cachear el modelo. Lo único que debemos especificar para el problema es la cantidad de etiquetas

In [31]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-ba

La advertencia dice que perdemos algunos pesos asociados a las capas de vocabulario y otras inicializadas aleatoriamente (capas del clasificador). Lo anterior es esperado, ya que estamos removiento las capas finales para ajustarlas con los datos que tenemos

Para instanciar un ```Trainer```, necesitamos definir 3 cosas más. La más importante son los ```TrainingArgumnts```, la cual es una clase que contiene todos los atributos para personalizar el entrenamiento. Ésta requiere un nombre de carpeta, el cual se usará para guardar los checkpoints del modelo y los otros argumentos son opcionales.

In [32]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

Aquí vemos que la evaluación se hace en cada epoch, se manipula la tasa de aprendizaje, se usa el tamaño del batch y se define el número de epochs para el entrenamiento, así como el decaimiento de los pesos.

Luego necesitaremos un ```data_collator``` para que tomen los ejemplos por lotes mientras le aplica padding del mismo tamaño (el padding será del largo del tamaño del más largo ejemplo). Existe un collator para esta tarea en la librería de Transformers, que no sólo aplica el padding sobre las entradas, pero también en las etiquetas:

In [33]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

Finalmente definimos la métrica sobre la cual se evaluará el entrenamiento, en este caso representada por el framework ```seqeval``` [documentación asociada](https://github.com/chakki-works/seqeval)

In [34]:
metric = load_metric("seqeval")

La métrica toma una lista de etiquetas para las predicciones y referencias

In [35]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

Necesitaremos post procesar las predicciones

* Seleccionar el índice predicho (salida con máximo logit) para cada token
* Convertir el token a su etiqueta en string asociada
* Ignorar todo lo que tenga la etiqueta -100

La siguiente función realiza todo este post procesamiento sobre el resultado ```Trainer.evaluate``` antes de aplicar la métrica

In [36]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Notar que se botan las métricas por etiqueta (la agregaremos luego) y se focalizan sobre los globales. Con lo anterior está listo para entregarlo al ```Trainer```

In [37]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.210600,0.146080,0.796673,0.774286,0.785320,0.957432
2,0.092000,0.137706,0.812020,0.830790,0.821298,0.962588
3,0.060100,0.143024,0.822331,0.835625,0.828925,0.964022


TrainOutput(global_step=1563, training_loss=0.11845848412370347, metrics={'train_runtime': 28845.8315, 'train_samples_per_second': 0.054, 'total_flos': 51706117436472.0, 'epoch': 3.0})

In [39]:
trainer.evaluate()

{'eval_loss': 0.1430237740278244,
 'eval_precision': 0.8223312518584597,
 'eval_recall': 0.8356247167245807,
 'eval_f1': 0.8289246908954664,
 'eval_accuracy': 0.9640221699481197,
 'eval_runtime': 621.9279,
 'eval_samples_per_second': 3.081,
 'epoch': 3.0}

In [40]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.8322932917316692,
  'recall': 0.8508771929824561,
  'f1': 0.8414826498422713,
  'number': 1254},
 'MISC': {'precision': 0.5792682926829268,
  'recall': 0.521978021978022,
  'f1': 0.5491329479768786,
  'number': 728},
 'ORG': {'precision': 0.8139450867052023,
  'recall': 0.8553530751708428,
  'f1': 0.8341355053683821,
  'number': 2634},
 'PER': {'precision': 0.9064356435643565,
  'recall': 0.9141288067898152,
  'f1': 0.9102659706686551,
  'number': 2003},
 'overall_precision': 0.8223312518584597,
 'overall_recall': 0.8356247167245807,
 'overall_f1': 0.8289246908954664,
 'overall_accuracy': 0.9640221699481197}